In [1]:
%load_ext autoreload
%autoreload 2
from notebook import *
# if get something about NUMEXPR_MAX_THREADS being set incorrectly, don't worry.  It's not a problem.

# Branch and branch predictions

## Why do we have "branches" in code?

Consider the following code snippet, how does the compiler translate to instructions?

In [4]:
render_code("branch.c", show="main")

// branch.c:26-45 (20 lines)
int main(int argc, char **argv)
{
    FILE *fin;
    int size,i;
    uint64_t *data;
    uint64_t threshold;
    fin = fopen(argv[1],"r");
    size = atoi(argv[2]);
    threshold = atoi(argv[3]);
    data = (uint64_t *)malloc(sizeof(uint64_t)*size);
    fread(data, sizeof(uint64_t),size,fin);
    for(i=0;i<size;i++)
    {
        if (data[i] % 2 != 0)
            call_when_true(&data[i]);
        else
            call_when_false(&data[i]);
    }
    return 0;
}

In [2]:
! gcc -S -O0 branch.c
render_code("branch.s", show=[".L12:",".LFE9:"])

branch.c:17:1: warning: ‘O0’ attribute directive ignored []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wattributes-Wattributes]8;;]
   17 | {
      | ^


// branch.s:169-206 (38 lines)
.L12:
	movl	-32(%rbp), %eax
	cltq
	leaq	0(,%rax,8), %rdx
	movq	-8(%rbp), %rax
	addq	%rdx, %rax
	movq	(%rax), %rax
	andl	$1, %eax
	testq	%rax, %rax
	je	.L10
	movl	-32(%rbp), %eax
	cltq
	leaq	0(,%rax,8), %rdx
	movq	-8(%rbp), %rax
	addq	%rdx, %rax
	movq	%rax, %rdi
	call	call_when_true
	jmp	.L11
.L10:
	movl	-32(%rbp), %eax
	cltq
	leaq	0(,%rax,8), %rdx
	movq	-8(%rbp), %rax
	addq	%rdx, %rax
	movq	%rax, %rdi
	call	call_when_false
.L11:
	addl	$1, -32(%rbp)
.L9:
	movl	-32(%rbp), %eax
	cmpl	-28(%rbp), %eax
	jl	.L12
	movl	$0, %eax
	leave
	.cfi_def_cfa 7, 8
	ret
	.cfi_endproc
.LFE9:

In [3]:
! gcc -S -O2 loop.c
render_code("loop.s", show=["loop1","LFE24"])

// loop.s:52-69 (18 lines)
	.globl	loop1
	.type	loop1, @function
loop1:
.LFB24:
	.cfi_startproc
	endbr64
	leaq	40(%rdi), %rdx
	xorl	%eax, %eax
	.p2align 4,,10
	.p2align 3
.L16:
	addl	(%rdi), %eax
	addq	$4, %rdi
	cmpq	%rdx, %rdi
	jne	.L16
	ret
	.cfi_endproc
.LFE24:

## Sorting and branch miss rates

Do you remember this?

In [8]:
compare([do_render_code("arraySort.cpp",show=["//START","//END"]),do_render_code("calculate_sum.c", show="calculate_sum")])

In [9]:
! lscpu | grep 'Model name'

Model name:                           Intel(R) Core(TM) i7-14700K


In [10]:
! make clean; make EXTRA_OPTS=-DCOUNT_SORTING; sleep 2
! echo "size,iterations,sorted,IC,Cycles,CPI,CT,ET,L1_dcache_miss_rate,L1_dcache_misses,L1_dcache_accesses,branches,branch_misses" > stats.csv
! echo -n "131072,1000,0," >> stats.csv
! taskset -c 8 ./arraySort 131072 1000 0
! echo -n "131072,1000,1," >> stats.csv
! taskset -c 8 ./arraySort 131072 1000 1

rm -f madd arraySort *.o
gcc -g -DHAVE_LINUX_PERF_EVENT_H -O0 -DCOUNT_SORTING -I/nfshome/htseng/courses/CS203/demo/branch  -o calculate_sum.o -c calculate_sum.c
gcc -g -DHAVE_LINUX_PERF_EVENT_H -O3 -DCOUNT_SORTING -I/nfshome/htseng/courses/CS203/demo/branch  -o perfstats.o -c perfstats.c
perfstats.c: In function ‘change_cpufrequnecy’:
perfstats.c:115:11: warning: implicit declaration of function ‘sched_getcpu’; did you mean ‘SYS_getcpu’? []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wimplicit-function-declaration-Wimplicit-function-declaration]8;;]
  115 |     cpu = sched_getcpu();
      |           ^~~~~~~~~~~~
      |           SYS_getcpu
g++ -O3 -DHAVE_LINUX_PERF_EVENT_H -DCOUNT_SORTING arraySort.cpp perfstats.o calculate_sum.o -o arraySort
arraySort.cpp: In function ‘int main(int, char**)’:
arraySort.cpp:53:22: warning: zero-length gnu_printf format string []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wformat-zero-length-Wformat-zero-

In [11]:
display_df_mono(render_csv("stats.csv"))

,index,size,iterations,sorted,IC,Cycles,CPI,CT,ET,L1_dcache_miss_rate,L1_dcache_misses,L1_dcache_accesses,branches,branch_misses
0,0,131072,1000,0,1509679743,1526078874,1.010863,0.179295,0.273619,0.001649,1944116,1178821893,262739750,35193548
1,1,131072,1000,1,1522725405,365287283,0.239890,0.179431,0.065544,0.002019,2387354,1182574195,266035194,962758


In [13]:
miss_predictions_diff = (35193548-962758)
cycles_diff = 1526078874-365287283
cycles_diff/miss_predictions_diff

33.91074500471652

What did we learn?

The CPI is ??? smaller with data sorted (including sorting itself)

The ET is ??? faster with data sorted (including sorting itself)

What's the cost of branch misses?

Let's exclude the sorting part and do it again.

In [15]:
! make clean; make
! sleep 2
! echo "size,iterations,sorted,IC,Cycles,CPI,CT,ET,L1_dcache_miss_rate,L1_dcache_misses,L1_dcache_accesses,branches,branch_misses" > stats.csv
! echo -n "262144,1000,0," >> stats.csv
! ./arraySort 262144 10000 0
! echo -n "262144,1000,1," >> stats.csv
! ./arraySort 262144 10000 1
display_df_mono(render_csv("stats.csv"))

rm -f madd arraySort *.o
gcc -g -DHAVE_LINUX_PERF_EVENT_H -O0  -I/nfshome/htseng/courses/CS203/demo/branch  -o calculate_sum.o -c calculate_sum.c
gcc -g -DHAVE_LINUX_PERF_EVENT_H -O3  -I/nfshome/htseng/courses/CS203/demo/branch  -o perfstats.o -c perfstats.c
perfstats.c: In function ‘change_cpufrequnecy’:
perfstats.c:115:11: warning: implicit declaration of function ‘sched_getcpu’; did you mean ‘SYS_getcpu’? []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wimplicit-function-declaration-Wimplicit-function-declaration]8;;]
  115 |     cpu = sched_getcpu();
      |           ^~~~~~~~~~~~
      |           SYS_getcpu
g++ -O3 -DHAVE_LINUX_PERF_EVENT_H  arraySort.cpp perfstats.o calculate_sum.o -o arraySort
arraySort.cpp: In function ‘int main(int, char**)’:
arraySort.cpp:53:22: warning: zero-length gnu_printf format string []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wformat-zero-length-Wformat-zero-length]8;;]
   53 |     sprintf(preamble,""

,index,size,iterations,sorted,IC,Cycles,CPI,CT,ET,L1_dcache_miss_rate,L1_dcache_misses,L1_dcache_accesses,branches,branch_misses
0,0,262144,1000,0,30211944185,30354275854,1.004711,0.179328,5.443376,0.001409,33289024,23617839741,5254231874,693976097
1,1,262144,1000,1,30162118030,6876129483,0.227972,0.179622,1.235103,0.001187,28019353,23599579307,5245555505,32229


In [16]:
Diff_Misses = 693976097-32229
Diff_Cycles = 30354275854-6876129483
print(Diff_Cycles/Diff_Misses)

33.83291855963197


Let's try a different processor!

In [19]:
! make clean; make
! sleep 2
! echo "size,iterations,sorted,IC,Cycles,CPI,CT,ET,L1_dcache_miss_rate,L1_dcache_misses,L1_dcache_accesses,branches,branch_misses" > stats.csv
! ssh htseng@blissey 'make -C /nfshome/htseng/courses/CSE142/demo/branch/ clean all ; lscpu|grep "Model name"'
! echo -n "262144,1000,0," >> stats.csv
! ssh htseng@blissey "cd /nfshome/htseng/courses/CSE142/demo/branch; ./arraySort 262144 1000 0"
! echo -n "262144,1000,1," >> stats.csv
! ssh htseng@blissey "cd /nfshome/htseng/courses/CSE142/demo/branch; ./arraySort 262144 1000 1"
display_df_mono(render_csv("stats.csv"))

rm -f madd arraySort *.o
gcc -g -DHAVE_LINUX_PERF_EVENT_H -O0  -I/nfshome/htseng/courses/CS203/demo/branch  -o calculate_sum.o -c calculate_sum.c
gcc -g -DHAVE_LINUX_PERF_EVENT_H -O3  -I/nfshome/htseng/courses/CS203/demo/branch  -o perfstats.o -c perfstats.c
perfstats.c: In function ‘change_cpufrequnecy’:
perfstats.c:115:11: warning: implicit declaration of function ‘sched_getcpu’; did you mean ‘SYS_getcpu’? []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wimplicit-function-declaration-Wimplicit-function-declaration]8;;]
  115 |     cpu = sched_getcpu();
      |           ^~~~~~~~~~~~
      |           SYS_getcpu
g++ -O3 -DHAVE_LINUX_PERF_EVENT_H  arraySort.cpp perfstats.o calculate_sum.o -o arraySort
arraySort.cpp: In function ‘int main(int, char**)’:
arraySort.cpp:53:22: warning: zero-length gnu_printf format string []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wformat-zero-length-Wformat-zero-length]8;;]
   53 |     sprintf(preamble,""

,index,size,iterations,sorted,IC,Cycles,CPI,CT,ET,L1_dcache_miss_rate,L1_dcache_misses,L1_dcache_accesses,branches,branch_misses
0,0,262144,1000,0,262144,1000,1,nan,nan,nan,nan,nan,nan,nan


In [33]:
Diff_Misses = 49159652-57646
Diff_Cycles = 1941551579-789330059
print(Diff_Cycles/Diff_Misses)

23.46587469359195


In [35]:
new_CPI = 0.9*1+0.1*24
print(new_CPI)

3.3000000000000003


In [46]:
! make clean; make EXTRA_OPTS=-O3; sleep 2
#! echo "size,iterations,sorted,IC,Cycles,CPI,CT,ET,L1_dcache_miss_rate,L1_dcache_misses,L1_dcache_accesses,branches,branch_misses" > stats.csv
! echo -n "131072,1000,2," >> stats.csv
! ./bstSearch 131072 1000 1
#! echo -n "131072,1000,1," >> stats.csv
#! ./bstSearch 131072 1000 1

rm -f madd arraySort *.o
gcc -g -DHAVE_LINUX_PERF_EVENT_H -O0 -O3 -I/nfshome/htseng/courses/CS203/demo/branch  -o calculate_sum.o -c calculate_sum.c
gcc -g -DHAVE_LINUX_PERF_EVENT_H -O3 -O3 -I/nfshome/htseng/courses/CS203/demo/branch  -o perfstats.o -c perfstats.c
perfstats.c: In function ‘change_cpufrequnecy’:
perfstats.c:115:11: warning: implicit declaration of function ‘sched_getcpu’; did you mean ‘SYS_getcpu’? []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wimplicit-function-declaration-Wimplicit-function-declaration]8;;]
  115 |     cpu = sched_getcpu();
      |           ^~~~~~~~~~~~
      |           SYS_getcpu
g++ -O3 -DHAVE_LINUX_PERF_EVENT_H -O3 arraySort.cpp perfstats.o calculate_sum.o -o arraySort
arraySort.cpp: In function ‘int main(int, char**)’:
arraySort.cpp:53:22: warning: zero-length gnu_printf format string []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wformat-zero-length-Wformat-zero-length]8;;]
   53 |     sprintf(pr

In [30]:
display_df_mono(render_csv("stats.csv"))

,index,size,iterations,sorted,IC,Cycles,CPI,CT,ET,L1_dcache_miss_rate,L1_dcache_misses,L1_dcache_accesses,branches,branch_misses
0,0,131072,1000,0,1509863524,1549902264,1.026518,0.202682,0.314137,0.003936,4640716,1178910227,262765964,35334854
1,1,131072,1000,1,1522994340,378758637,0.248693,0.195570,0.074074,0.006281,7428313,1182686456,266077862,963615
2,2,131072,1000,2,1968171657,460758914,0.234105,0.192836,0.088851,0.111585,73230012,656273699,524735930,4080


In [33]:
! make clean; make
! sleep 2
! echo "size,iterations,sorted,IC,Cycles,CPI,CT,ET,L1_dcache_miss_rate,L1_dcache_misses,L1_dcache_accesses,branches,branch_misses" > stats.csv
! echo -n "131072,2000,0," >> stats.csv
! ./arraySort 131072 2000 0
! echo -n "131072,2000,1," >> stats.csv
! ./arraySort 131072 2000 1
! echo -n "131072,2000,2," >> stats.csv
! ./bstSearch 131072 2000 1
! echo -n "131072,8000,0," >> stats.csv
! ./arraySort 131072 8000 0
! echo -n "131072,8000,1," >> stats.csv
! ./arraySort 131072 8000 1
! echo -n "131072,8000,2," >> stats.csv
! ./bstSearch 131072 8000 1
! echo -n "262144,8000,0," >> stats.csv
! ./arraySort 131072 8000 0
! echo -n "262144,8000,1," >> stats.csv
! ./arraySort 131072 8000 1
! echo -n "262144,8000,2," >> stats.csv
! ./bstSearch 131072 8000 1
display_df_mono(render_csv("stats.csv"))

rm -f madd arraySort *.o
gcc -g -DHAVE_LINUX_PERF_EVENT_H -O0  -I/nfshome/htseng/courses/CS203/demo/branch  -o calculate_sum.o -c calculate_sum.c
gcc -g -DHAVE_LINUX_PERF_EVENT_H -O3  -I/nfshome/htseng/courses/CS203/demo/branch  -o perfstats.o -c perfstats.c
perfstats.c: In function ‘change_cpufrequnecy’:
perfstats.c:115:11: warning: implicit declaration of function ‘sched_getcpu’; did you mean ‘SYS_getcpu’? []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wimplicit-function-declaration-Wimplicit-function-declaration]8;;]
  115 |     cpu = sched_getcpu();
      |           ^~~~~~~~~~~~
      |           SYS_getcpu
g++ -O3 -DHAVE_LINUX_PERF_EVENT_H  arraySort.cpp perfstats.o calculate_sum.o -o arraySort
arraySort.cpp: In function ‘int main(int, char**)’:
arraySort.cpp:53:22: warning: zero-length gnu_printf format string []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wformat-zero-length-Wformat-zero-length]8;;]
   53 |     sprintf(preamble,""

,index,size,iterations,sorted,IC,Cycles,CPI,CT,ET,L1_dcache_miss_rate,L1_dcache_misses,L1_dcache_accesses,branches,branch_misses
0,0,131072,2000,0,2984770554,3007203279,1.007516,0.201416,0.605699,0.001849,4311738,2331833932,519227293,68647965
1,1,131072,2000,1,3015204632,698187494,0.231556,0.194408,0.135733,0.000844,1988892,2357737595,524583833,6356
2,2,131072,2000,2,6823832406,1744884131,0.255704,0.193966,0.338449,0.002207,11578272,5246442261,1050300896,11092
3,3,131072,8000,0,12071727068,12237438812,1.013727,0.195507,2.392503,0.001563,14748940,9435044957,2099612688,280586309
4,4,131072,8000,1,12063724493,2804339648,0.232461,0.194703,0.546014,0.006951,65601263,9437171232,2098287874,25243
5,5,131072,8000,2,27295069760,7036781914,0.257804,0.193147,1.359130,0.003060,64226511,20985630184,4201169491,34202
6,6,262144,8000,0,12076679787,12273586044,1.016305,0.194183,2.383325,0.001425,13455311,9439409418,2100494688,281188135
7,7,262144,8000,1,12015017752,2778105044,0.231219,0.195055,0.541883,0.002974,27953942,9399400096,2089763454,24232
8,8,262144,8000,2,27295112959,7010649994,0.256846,0.193507,1.356607,0.002369,49706982,20985657463,4201171869,36024
